In [32]:
import json
from getpass import getpass
import requests
import sys
import time
import cgi
import os
import pandas as pd
import geopandas as gpd
import warnings
warnings.filterwarnings("ignore")
apiKey = "4oCpGCCrIET5H@Cd"

In [33]:
# Send http request
def sendRequest(url, data, apiKey = None, exitIfNoResponse = True):
    """
    Send a request to an M2M endpoint and returns the parsed JSON response.

    Parameters:
    endpoint_url (str): The URL of the M2M endpoint
    payload (dict): The payload to be sent with the request

    Returns:
    dict: Parsed JSON response
    """
      
    json_data = json.dumps(data)
    
    if apiKey == None:
        response = requests.post(url, json_data)
    else:
        headers = {'X-Auth-Token': apiKey}              
        response = requests.post(url, json_data, headers = headers)  
    
    try:
      httpStatusCode = response.status_code 
      if response == None:
          print("No output from service")
          if exitIfNoResponse: sys.exit()
          else: return False
      output = json.loads(response.text)
      if output['errorCode'] != None:
          print(output['errorCode'], "- ", output['errorMessage'])
          if exitIfNoResponse: sys.exit()
          else: return False
      if  httpStatusCode == 404:
          print("404 Not Found")
          if exitIfNoResponse: sys.exit()
          else: return False
      elif httpStatusCode == 401: 
          print("401 Unauthorized")
          if exitIfNoResponse: sys.exit()
          else: return False
      elif httpStatusCode == 400:
          print("Error Code", httpStatusCode)
          if exitIfNoResponse: sys.exit()
          else: return False
    except Exception as e: 
          response.close()
          print(e)
          if exitIfNoResponse: sys.exit()
          else: return False
    response.close()
    
    return output['data']

In [34]:
def downloadfiles(downloadIds):
    downloadIds.append(download['downloadId'])
    print("    DOWNLOADING: " + download['url'])
    downloadResponse = requests.get(download['url'], stream=True)

    # parse the filename from the Content-Disposition header
    content_disposition = cgi.parse_header(downloadResponse.headers['Content-Disposition'])[1]
    filename = os.path.basename(content_disposition['filename'])
    filepath = os.path.join(data_dir, filename)

    # write the file to the destination directory
    with open(filepath, 'wb') as f:
        for data in downloadResponse.iter_content(chunk_size=8192):
            f.write(data)
    #print(f"    DOWNLOADED {filename} ({i+1}/{len(downloadIds)})")

In [35]:
data_dir = 'data'
utils_dir = 'utils'
dirs = [ data_dir, utils_dir]

for d in dirs:
        if not os.path.exists(d): 
            try: 
                os.makedirs(d)
                print(f"Directory '{d}' created successfully.") 
            except OSError as e: 
                print(f"Error creating directory '{d}': {e}") 
        else: 
            print(f"Directory '{d}' already exists.") 


Directory 'data' already exists.
Directory 'utils' already exists.


In [36]:
username = "Irving2209"
token = "bzkI06wtA!tXNxkD7FubEVhFS2O80BSxZToZTBr6QtBg5ojEzTxes7c5FXDI7LXK"

In [37]:
print("Logging in...\n")
    
serviceUrl = "https://m2m.cr.usgs.gov/api/api/json/stable/"
login_payload = {'username' : username, 'token' : token}
    
apiKey = sendRequest(serviceUrl + "login-token", login_payload)
    
print("API Key: " + apiKey + "\n")

Logging in...

API Key: eyJjaWQiOjI2OTI0MTc4LCJzIjoiMTc1OTQzMDk5MSIsInIiOjk0NywicCI6WyJ1c2VyIiwiZG93bmxvYWQiLCJvcmRlciJdfQ==



## **1. Create an Area of Interest (AOI) GeoJSON Text File** <a id="createaoi"></a>

Polygon coordinates are arranged in the sequence NE, NW, SW, SE, NE, representing all four corners, with an extra NE point to conclude and close the polygon.

In [38]:
from geojson import Polygon, Feature, FeatureCollection, dump

bounding_box = [[-57.7448764485,-25.4967482196],  # Lower left (min longitude, min latitude)
                [-57.809500565,-25.2208058563],  # Lower right (max longitude, min latitude)
                [-57.4572636009,-25.1520434564],  # Upper right (max longitude, max latitude)
                [-57.3713346464,-25.4486640975],  # Upper left (min longitude, max latitude)
                [-57.7448764485,-25.4967482196]] # Closing the polygon (back to lower left)

polygon = Polygon([bounding_box])

features = []
features.append(Feature(geometry=polygon, properties={"region": "Panama Centro"}))

feature_collection = FeatureCollection(features)

with open('./utils/panama_centro.geojson', 'w') as f:
    dump(feature_collection, f)

In [39]:
aoi_geodf = gpd.read_file('./utils/panama_centro.geojson')

- ### View the Coordinate Reference System (CRS)

> The CRS used is the World Geodetic System 1984 with units in degrees.

In [40]:
aoi_geodf.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [41]:
aoi_geodf.explore()

- ### Plot AOI using Folium

> The ***folium*** module is used for creating interactive web maps.

## **2. Submit a [*dataset-search*](https://m2m.cr.usgs.gov/api/docs/reference/#dataset-search)**<a id="dataset-search"></a>

The [***dataset-search***](https://m2m.cr.usgs.gov/api/docs/reference/#dataset-search) is used to find available collections of datasets. Users can pass the API without search parameters to view all the available datasets; however, in this case, we use the ***'datasetName'*** input parameter to limit the search to find datasets with ***'Landsat 8-9'*** since Landsat-8 and Landsat-9 datasets are grouped.  

- ### Setup search  parameters into a ***payload*** used for sending requests:

> Adding a [***temporal-filter***](https://m2m.cr.usgs.gov/api/docs/datatypes/#temporalFilter) will return data collections available within a set period of time. The ***'start'*** and ***'end'*** dates in ISO format (**yyyy-mm-dd**) for the time period of interest.

In [45]:
temporalFilter = {'start' : '2025-09-05', 'end' : '2025-10-01'}

- ### Include ***'Landsat 8-9'*** as ***'datasetName'***:

> Note that ***'Landsat 8-9'***  is not a ***'datasetName'*** but can be used to query for specifically Landsat-8 or Landsat-9 data.

In [46]:
datasearch_payload = {'datasetName': 'Landsat 8-9',
                     'temporalFilter' : temporalFilter}

In [47]:
datasearch_result = sendRequest(serviceUrl + "dataset-search", datasearch_payload, apiKey)

<div class="alert alert-info">
    <h4>To find the <b><i>datasetName</i></b> for other collections of Landsat data:</h4>
    <ol>
        <li>
            Run a request with the <a href="https://m2m.cr.usgs.gov/api/docs/reference/#dataset-search" target="_blank">
                <b><i>dataset-search</i></b>
            </a> request and extract the <b><i>'datasetAlias'</i></b> field. No input parameters are necessary!
        </li>
        <li>
            Run the <b><i>dataset-search</i></b> endpoint using the
            <a href="https://m2m.cr.usgs.gov/api/test/json/" target="_blank">
                <b><i>Machine-to-Machine (M2M) Test Page</i></b>.
            </a>
        </li>
    </ol>
</div>


- ### Visualize results using *pandas*:

In [48]:
pd.json_normalize(datasearch_result)

,abstractText,acquisitionStart,acquisitionEnd,catalogs,collectionName,collectionLongName,datasetId,datasetAlias,datasetCategoryName,dataOwner,...,legacyId,sceneCount,temporalCoverage,supportCloudCover,supportDeletionSearch,allowInKmz,spatialBounds.north,spatialBounds.east,spatialBounds.south,spatialBounds.west
0,The USGS Earth Resources Observation and Scien...,2013-04-11,None,"[EE, GV]",Landsat 8-9 OLI/TIRS C2 L1,Landsat 8-9 Operational Land Imager and Therma...,5e81f14f59432a27,landsat_ot_c2_l1,Landsat Collection 2 Level-1,LSAA,...,9825,4298995,"[""2013-03-08 00:00:00-06"",""2025-09-29 00:00:00...",True,True,True,84.441827,197.576528,-84.571377,-180.009967
1,The USGS Earth Resources Observation and Scien...,2013-04-11,None,[EE],Landsat 8-9 OLI/TIRS C2 L2,Landsat 8-9 Operational Land Imager and Therma...,5e83d14f2fc39685,landsat_ot_c2_l2,Landsat Collection 2 Level-2,LSAA,...,9885,3714007,"[""2013-03-18 00:00:00-05"",""2025-09-27 00:00:00...",True,True,True,84.441827,194.444152,-84.517917,-180.009967


## **3. Send a [*dataset-filters*](https://m2m.cr.usgs.gov/api/docs/reference/#dataset-filters) Request** <a id="dataset-filters"></a>
The [***dataset-filters***](https://m2m.cr.usgs.gov/api/docs/reference/#dataset-filters) request returns the [***metadataFilter***](https://m2m.cr.usgs.gov/api/docs/datatypes/#metadataFilter) parameters for the specified dataset. These values can be used as additional criteria when submitting search queries for specific scenes. The ***'datasetName'*** input parameter is necessary.  

> Note that the Landsat-9 Collection 2 Level 2 datasets has the ***dataAlias:*** ***'landsat_ot_c2_l2'***

In [49]:
datasetName = datasearch_result[0]['datasetAlias']

In [50]:
datafilter_payload = {'datasetName': datasetName}
datafilter_result = sendRequest(serviceUrl + "dataset-filters", datafilter_payload, apiKey)

> Below we plot the results from the [***dataset-filters***](https://m2m.cr.usgs.gov/api/docs/reference/#dataset-filters) query. The **`fieldLabel`** provides insights into the queryable fields available for this specific Landsat data collection. While the **`fieldConfig.type`** that lists types `Text`, `Range` and `Select` guides users on the appropriate [***metadataFilter***](https://m2m.cr.usgs.gov/api/docs/datatypes/#metadataFilter) type to apply.
    
> **Example:**
> - If type is `Text`, then use a `'like'` or `'='` operand in a `'value'` metadataFilter
> - If type is `Select`, then check the `valueList` for valid values to use in a `'value'` metadataFilter (use the key on the left, not the label on the right)
> - If type is `Range`, then use a `'between'` metadataFilter and provide `'firstValue'` and `'secondValue'`

In [51]:
pd.json_normalize(datafilter_result)

,id,legacyFieldId,dictionaryLink,fieldLabel,searchSql,fieldConfig.type,fieldConfig.filters,fieldConfig.options.size,fieldConfig.validators,fieldConfig.numElements,...,valueList.OFFNADIR,valueList.L1TP,valueList.L1GT,valueList.L1GS,valueList.T1,valueList.T2,valueList.RT,fieldConfig.options.size,valueList.0,valueList.-1
0,5e81f14fe3c40983,27678.0,https://www.usgs.gov/centers/eros/science/land...,Landsat Product Identifier L1,LANDSAT_PRODUCT_ID_L1 like ?,Text,"[{'type': 'Application\Filter\Like', 'options'...",45,[],5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,5e81f14f8faf8048,27679.0,https://www.usgs.gov/centers/eros/science/land...,WRS Path,WRS_PATH between ?,Range,[],NaN,[],1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,5e81f14f8d2a7c24,27680.0,https://www.usgs.gov/centers/eros/science/land...,WRS Row,WRS_ROW between ?,Range,[],NaN,[],1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,61af93b8fad2acf5,NaN,https://www.usgs.gov/centers/eros/science/land...,Satellite,satellite =?,Select,[],NaN,[],NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5e81f14f85d499dc,27685.0,https://www.usgs.gov/centers/eros/science/land...,Sensor Identifier,SENSOR_ID = ?,Select,"[{'type': 'StringToUpper', 'options': []}]",4,[],1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,5e81f14f61bda7c4,27686.0,https://www.usgs.gov/centers/eros/science/land...,Day/Night Indicator,DAY_NIGHT = ?,Select,"[{'type': 'StringToUpper', 'options': []}]",3,[],1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,5e81f150e42bc489,27690.0,https://www.usgs.gov/centers/eros/science/land...,Nadir/Off Nadir,NADIR_OFFNADIR = ?,Select,"[{'type': 'StringToUpper', 'options': []}]",3,[],1,...,Off Nadir,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,5e81f150c2f58f31,27790.0,https://www.usgs.gov/centers/eros/science/land...,Sun Elevation L0RA,SUN_ELEVATION_L0RA between ?,Range,[],NaN,[],1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,5e81f14fcf660794,27684.0,https://www.usgs.gov/centers/eros/science/land...,Data Type L1,"RIGHT(DATA_TYPE_L1, 4) = ?",Select,[],4,[],1,...,NaN,Level 1TP,Level 1GT,Level 1GS,NaN,NaN,NaN,NaN,NaN,NaN
9,5e81f14fff5055a3,27683.0,https://www.usgs.gov/centers/eros/science/land...,Collection Category,COLLECTION_CATEGORY = ?,Select,[],4,[],1,...,NaN,NaN,NaN,NaN,Tier 1,Tier 2,Real-Time,NaN,NaN,NaN


## **4. Setup *scene-filters* and to submit a *scene-search*:**<a id="scene-search"></a>

- ### Setup [***Scene-filters***](https://m2m.cr.usgs.gov/api/docs/datatypes/#sceneFilter):

> The [***spatialFilter***](https://m2m.cr.usgs.gov/api/docs/datatypes/#spatialFilter), [***acquisitionFilter***](https://m2m.cr.usgs.gov/api/docs/datatypes/#acquisitionFilter), [**metadataFilter**](https://m2m.cr.usgs.gov/api/docs/datatypes/#metadataFilter) and [***cloudCoverFilter***](https://m2m.cr.usgs.gov/api/docs/datatypes/#cloudCoverFilter) are [***scene-filters***](https://m2m.cr.usgs.gov/api/docs/datatypes/#sceneFilter) used below to identify the scenes matching the filter criteria using [**scene-search**](https://m2m.cr.usgs.gov/api/docs/reference/#scene-search).
>
> #### [***spatialFilter***](https://m2m.cr.usgs.gov/api/docs/datatypes/#spatialFilter)
>
> There are two spatial ***filterType's***:
>
> 1. [***geojson***](https://m2m.cr.usgs.gov/api/docs/datatypes/#spatialFilterGeoJson) which needs a ***['geoJson'](https://m2m.cr.usgs.gov/api/docs/datatypes/#geoJson)*** field name labeled with a geometric ***'type'*** e.g. ***'Polygon'*** and includes the coordinate array of the polygon:
>
> ~~~~
>        spatialFilter = {'filterType' : 'geojson',
>                 'geoJson' : {'type': 'Polygon',\ 
>                              'coordinates': [[[-75.446, 36.3793],\
>                                                                  [-76.9196, 36.3793],\
>                                                                  [-76.9196, 35.5713],
>                                                                  [-75.446, 35.5713],\
>                                                                  [-75.446, 36.3793]]]}}
>
> ~~~~
>
> 2. [***mbr***](https://m2m.cr.usgs.gov/api/docs/datatypes/#spatialFilterMbr) (Minimum Bounding Rectangle) where the user should include the ***'lowerLeft'*** (southwest point) and ***'upperRight'*** (northeast point) of the rectangle as ***{'latitude': ######, 'longitude' : ######}*** coordinates:
> 
> ```
>        spatialFilter =  {'filterType' : 'mbr',
>                      'lowerLeft' : {'latitude' : 35.5713,\
>                                     'longitude' : -76.9196},
>                     'upperRight' : { 'latitude' : 36.3793,\
>                                     'longitude' : -75.446}}
> ```

In [62]:
spatialFilter =  {'filterType' : 'mbr',
                    'lowerLeft' : {'latitude' : float(aoi_geodf.bounds.miny[0]),\
                                   'longitude' : float(aoi_geodf.bounds.minx[0])},
                   'upperRight' : { 'latitude' : float(aoi_geodf.bounds.maxy[0]),\
                                   'longitude' : float(aoi_geodf.bounds.maxx[0])}}

# spatialFilter = {'filterType' : 'geojson',
#                  'geoJson' : {'type': 'Polygon',\ 
#                               'coordinates': [[[-57.7448764485,-25.4967482196],\
#                                                                   [-57.809500565,-25.2208058563],\
#                                                                   [-57.4572636009,-25.1520434564],
#                                                                   [-57.3713346464,-25.4486640975],\
#                                                                   [-57.7448764485,-25.4967482196]]]}}

> #### [***acquisitionFilter***](https://m2m.cr.usgs.gov/api/docs/datatypes/#acquisitionFilter) 
> The [***acquisitionFilter***](https://m2m.cr.usgs.gov/api/docs/datatypes/#acquisitionFilter) is the ***'start'*** and ***'end'*** dates in ISO format for the time period of interest. In our case it will be the same as the ***temporalFilter*** used above. It is also in ISO format **yyyy-mm-dd**. 

In [63]:
temporalFilter = {'start' : '2025-09-05', 'end' : '2025-10-01'}

> #### [***metadataFilter***](https://m2m.cr.usgs.gov/api/docs/datatypes/#metadataFilter)
> The [***metadataFilter***](https://m2m.cr.usgs.gov/api/docs/datatypes/#metadataFilter) is used to narrow down the scene search to find specific scenes e.g. in our case searching only for Landsat-9 images. A ***'filterId'*** (from the [***dataset-filters***](https://m2m.cr.usgs.gov/api/docs/reference/#dataset-filters) result), ***'filterType'*** and ***'value'*** (9 for Landsat-9) are necessary when searching for datasets from a specific satellite.

In [80]:
filterId = datafilter_result[3]['id'] #because the id for the satellite metadata filter
#filterId = datasearch_result[1]['datasetId']#gives error: Expecting ',' delimiter: line 1 column 163 (char 162)
metadataFilter = {'filterType': 'value',
                    'filterId': filterId,
                    'value' : '8'} # Filtro del satelite 8 o 9

> #### [***cloudCoverFilter***](https://m2m.cr.usgs.gov/api/docs/datatypes/#cloudCoverFilter) 
> To filter scenes by cloud cover, you can use the [***cloudCoverFilter***](https://m2m.cr.usgs.gov/api/docs/datatypes/#cloudCoverFilter) and specify a ***'min'*** and ***'max'*** percentage (%) value..

In [79]:
cloudCoverFilter = {'min' : 0, 'max' : 15}

- ### Combine all parameters into a *payload* used for sending requests:

In [66]:
search_payload = {
    'datasetName' : datasetName,
    'sceneFilter' : {
        'metadataFilter' : metadataFilter,
        'spatialFilter' : spatialFilter,
        'acquisitionFilter' : temporalFilter,
        'cloudCoverFilter' : cloudCoverFilter
    }
}

In [57]:
search_payload

{'datasetName': 'landsat_ot_c2_l1',
 'sceneFilter': {'metadataFilter': {'filterType': 'value',
   'filterId': '61af93b8fad2acf5',
   'value': '8'},
  'spatialFilter': {'filterType': 'mbr',
   'lowerLeft': {'latitude': -25.496748, 'longitude': -57.809501},
   'upperRight': {'latitude': -25.152043, 'longitude': -57.371335}},
  'acquisitionFilter': {'start': '2025-03-20', 'end': '2025-05-01'},
  'cloudCoverFilter': {'min': 0, 'max': 20}}}

- ### Submit a [***scene-search***](https://m2m.cr.usgs.gov/api/docs/reference/#scene-search)


> The [***scene-search***](https://m2m.cr.usgs.gov/api/docs/reference/#scene-search) is used for searching for scenes within a dataset collection, a ***'datasetName'*** parameter is required.

In [67]:
scenes = sendRequest(serviceUrl + "scene-search", search_payload, apiKey) 

In [68]:
results = pd.json_normalize(scenes['results'])
results_gdf = gpd.GeoDataFrame(results)
results_gdf.head()

,browse,cloudCover,entityId,displayId,orderingId,metadata,hasCustomizedMetadata,publishDate,options.bulk,options.download,...,options.secondary,selected.bulk,selected.compare,selected.order,spatialBounds.type,spatialBounds.coordinates,spatialCoverage.type,spatialCoverage.coordinates,temporalCoverage.endDate,temporalCoverage.startDate
0,"[{'id': '5e81f7d39594374', 'browseRotationEnab...",7,LC82260772025262LGN00,LC08_L1TP_226077_20250919_20250929_02_T1,None,"[{'id': '5e81f1503645de13', 'fieldName': 'ID',...",None,2025-09-19 11:30:19-05,True,True,...,False,False,False,False,Polygon,"[[[-58.82677, -25.60513], [-58.82677, -23.5027...",Polygon,"[[[-58.82677, -25.22535], [-56.9616, -25.60513...",2025-09-19 00:00:00,2025-09-19 00:00:00


# Visualización de los footprints de las imàgenes a descargar.

In [69]:
results_gdf["geometry"] = results_gdf['spatialBounds.coordinates'].apply(lambda x : Polygon(x))
results_gdf = results_gdf.set_crs(crs="EPSG:4326")


In [70]:
results_gdf.explore()

- ### Create a list of ***SceneIds***

> The ***entityIds*** are [Landsat Scene Identifiers](https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#landsat_scene_id) that remain consistent for all levels and products available for each scene. Below we group all entityIds into a variable called *`sceneIds`*.

In [71]:
sceneIds = []
for result in scenes['results']:
    # Add this scene to the list I would like to download
    sceneIds.append(result['entityId'])
    
sceneIds

['LC82260772025262LGN00']

## **5. Find the [***download-options***](https://m2m.cr.usgs.gov/api/docs/reference/#download-options) for the scenes**<a id="download-options"></a>

The [***download-options***](https://m2m.cr.usgs.gov/api/docs/reference/#download-options) request is used to discover downloadable products for each dataset. If a download is marked as not available, an order must be placed to generate that product. A ***'datasetName'*** parameter is required.

In [77]:
download_payload = {'datasetName' : datasetName, 
                    'entityIds' : sceneIds}

downloadOptions = sendRequest(serviceUrl + "download-options", download_payload, apiKey)


In [78]:
pd.json_normalize(downloadOptions)

,id,downloadName,displayId,entityId,datasetId,available,filesize,productName,productCode,bulkAvailable,downloadSystem,secondaryDownloads,fileGroups,checksum
0,632211e26883b1f7,,LC08_L1TP_226077_20250919_20250929_02_T1,LC82260772025262LGN00,5e81f14f59432a27,True,1200673331,Landsat Collection 2 Level-1 Product Bundle,D805,True,dds_ms,"[{'id': '5e8207607217ef3e', 'downloadName': ''...",None,NaN
1,73ceb05468b7e8c2,C2L1 Tile Product Files,LC08_L1TP_226077_20250919_20250929_02_T1,LC82260772025262LGN00,5e81f14f59432a27,True,0,Landsat Collection 2 Level-1 Band File,D687,True,folder,"[{'id': '5e8207607217ef3e', 'downloadName': ''...",None,"[{'id': 'sha512', 'value': None}]"
2,6447cc2e2a09aab6,C2L1 Tile Product Files,LC08_L1TP_226077_20250919_20250929_02_T1,LC82260772025262LGN00,5e81f14f59432a27,True,0,Landsat Collection 2 Level-1 Band File,D689,True,folder,"[{'id': '5e8207607217ef3e', 'downloadName': ''...",None,"[{'id': 'sha512', 'value': None}]"
3,5e81f14f92acf9ef,,LC08_L1TP_226077_20250919_20250929_02_T1,LC82260772025262LGN00,5e81f14f59432a27,False,1200673331,Landsat Collection 2 Level-1 Product Bundle,D690,False,ls_zip,"[{'id': '5e8207607217ef3e', 'downloadName': ''...",None,NaN
4,5e9eb01274d7924f,Full Resolution Browse (Reflective Color) GeoTIFF,LC08_L1TP_226077_20250919_20250929_02_T1,LC82260772025262LGN00,5e81f14f59432a27,True,14680064,Full-Resolution Browse (Natural Color) GeoTIFF,D734,True,ls_frb,[],None,NaN
5,5e9eb12795d8231b,,LC08_L1TP_226077_20250919_20250929_02_T1,LC82260772025262LGN00,5e81f14f59432a27,True,14680064,Full-Resolution Browse (Thermal) GeoTIFF,D735,True,ls_frb,[],None,NaN
6,5e9eb2249228fe8f,None,LC08_L1TP_226077_20250919_20250929_02_T1,LC82260772025262LGN00,5e81f14f59432a27,True,14680064,Full-Resolution Browse (Quality) GeoTIFF,D736,True,ls_frb,[],None,NaN
7,5f6184a83f05bbf9,Full Resolution Browse (Reflective Color) JPEG,LC08_L1TP_226077_20250919_20250929_02_T1,LC82260772025262LGN00,5e81f14f59432a27,True,6291456,Full-Resolution Browse (Natural Color) JPEG,D731,True,ls_frb,[],None,NaN
8,5f618512b2d7b4be,None,LC08_L1TP_226077_20250919_20250929_02_T1,LC82260772025262LGN00,5e81f14f59432a27,True,6291456,Full-Resolution Browse (Thermal) JPEG,D732,True,ls_frb,[],None,NaN
9,5f618552dc40c587,None,LC08_L1TP_226077_20250919_20250929_02_T1,LC82260772025262LGN00,5e81f14f59432a27,True,6291456,Full-Resolution Browse (Quality) JPEG,D733,True,ls_frb,[],None,NaN


<div class="alert alert-block alert-warning">
    <h4> <b>NOTE:</b> The scene list cannot exceed 50,000 items. </h4>
</div>  

- ### Create a list of available products using results from [***download-options***](https://m2m.cr.usgs.gov/api/docs/reference/#download-options)

> Make sure the product is available for this scene and ignore `'folders'` since they are tile files:

In [74]:
availableproducts = []
for product in downloadOptions:
        # Make sure the product is available for this scene
        if product['available'] == True and product['downloadSystem'] != 'folder':
                availableproducts.append({'entityId' : product['entityId'],
                                   'productId' : product['id']})

In [75]:
availableproducts

[{'entityId': 'LC82260772025262LGN00', 'productId': '632211e26883b1f7'},
 {'entityId': 'LC82260772025262LGN00', 'productId': '5e9eb01274d7924f'},
 {'entityId': 'LC82260772025262LGN00', 'productId': '5e9eb12795d8231b'},
 {'entityId': 'LC82260772025262LGN00', 'productId': '5e9eb2249228fe8f'},
 {'entityId': 'LC82260772025262LGN00', 'productId': '5f6184a83f05bbf9'},
 {'entityId': 'LC82260772025262LGN00', 'productId': '5f618512b2d7b4be'},
 {'entityId': 'LC82260772025262LGN00', 'productId': '5f618552dc40c587'}]

## **6. Submit a [*download-request*](https://m2m.cr.usgs.gov/api/docs/reference/#download-request)**<a id="download-request"></a>

The [**download-request**](https://m2m.cr.usgs.gov/api/docs/reference/#download-request) is used to request download urls and adds them to a queue.

In [62]:
requestedDownloadsCount = len(availableproducts)

# set a label for the download request
label = "download-sample"
download_req_payload = {'downloads' : availableproducts,
                                 'label' : label}

requestResults = sendRequest(serviceUrl + "download-request", download_req_payload, apiKey)
requestResults

{'availableDownloads': [{'downloadId': 790721113,
   'eulaCode': None,
   'entityId': 'LC80120542025091LGN00',
   'url': 'https://landsatlook.usgs.gov/gen-bundle?landsat_product_id=LC08_L1TP_012054_20250401_20250411_02_T1&requestSignature=eyJkb3dubG9hZEFwcCI6Ik0yTSIsImNvbnRhY3RJZCI6MjY5MjQxNzgsImRvd25sb2FkSWQiOjc5MDcyMTExMywiZGF0ZUdlbmVyYXRlZCI6IjIwMjUtMDUtMDJUMDA6NTY6NTUtMDU6MDAiLCJpZCI6IkxDMDhfTDFUUF8wMTIwNTRfMjAyNTA0MDFfMjAyNTA0MTFfMDJfVDEiLCJzaWduYXR1cmUiOiIkNSQkWFdUWUZRbVZsY29LXC83MU9ycHdxSWZFdFNSSEhNV3A0bFlVSHFKVVN1ejkifQ=='},
  {'downloadId': 790721114,
   'eulaCode': None,
   'entityId': 'LC80120542025091LGN00',
   'url': 'https://landsatlook.usgs.gov/gen-browse?size=frb&type=refl&product_id=LC08_L1TP_012054_20250401_20250411_02_T1&requestSignature=eyJkb3dubG9hZEFwcCI6Ik0yTSIsImNvbnRhY3RJZCI6MjY5MjQxNzgsImRvd25sb2FkSWQiOjc5MDcyMTExNCwiZGF0ZUdlbmVyYXRlZCI6IjIwMjUtMDUtMDJUMDA6NTY6NTUtMDU6MDAiLCJpZCI6IkxDMDhfTDFUUF8wMTIwNTRfMjAyNTA0MDFfMjAyNTA0MTFfMDJfVDEiLCJzaWduYXR1cmUiOiIkNSQkW

## **7. Submit a [*download-retrieve*](https://m2m.cr.usgs.gov/api/docs/reference/#download-retrieve)**<a id="download-retrieve"></a>

Here we call the [***download-retrieve***](https://m2m.cr.usgs.gov/api/docs/reference/#download-retrieve) method to get products that are not immediately available for download, these are listed under the **`'preparingDownloads'`** results after sending a ***download-request***. The download links that are immediately available are retrieved from the **`'availableDownloads'`** list.

In [64]:
 if requestResults['preparingDownloads'] != None and len(requestResults['preparingDownloads']) > 0:
    download_retrieve_payload = {'label' : label}
    
    print("Requesting for additional available download urls...")
    moreDownloadUrls = sendRequest(serviceUrl + "download-retrieve", download_retrieve_payload, apiKey)

    downloadIds = []  



    print("\nDownloading from available downloads:")    
    for download in moreDownloadUrls['available']:
        if str(download['downloadId']) in requestResults['newRecords'] or str(download['downloadId']) in requestResults['duplicateProducts']:
            downloadfiles(downloadIds)




    print("\nDownloading from requested downloads:")        
    for download in moreDownloadUrls['requested']:
        if str(download['downloadId']) in requestResults['newRecords'] or str(download['downloadId']) in requestResults['duplicateProducts']:
            downloadfiles(downloadIds)               



    # Didn't get all of the reuested downloads, call the download-retrieve method again probably after 30 seconds
    while len(downloadIds) < (requestedDownloadsCount - len(requestResults['failed'])): 
        preparingDownloads = requestedDownloadsCount - len(downloadIds) - len(requestResults['failed'])
        print("    ", preparingDownloads, " downloads are not available. Waiting for 30 seconds...")
        time.sleep(30)
        print("    Trying to retrieve data after waiting for 30 seconds...")
        moreDownloadUrls = sendRequest(serviceUrl + "download-retrieve", download_retrieve_payload, apiKey)
        for download in moreDownloadUrls['available']:                            
            if download['downloadId'] not in downloadIds and (str(download['downloadId']) in requestResults['newRecords'] or str(download['downloadId']) in requestResults['duplicateProducts']):
                downloadfiles(downloadIds)


else:
    print("\nAll downloads are available to download. Retrieving...\n")# Get all available downloads
    i = 0
    for download in requestResults['availableDownloads']:
        
        print("DOWNLOADING: " + download['url'])
        
        downloadResponse = requests.get(download['url'], stream=True)

        # parse the filename from the Content-Disposition header
        content_disposition = cgi.parse_header(downloadResponse.headers['Content-Disposition'])[1]
        filename = os.path.basename(content_disposition['filename'])
        filepath = os.path.join(data_dir, filename)

        # write the file to the destination directory
        with open(filepath, 'wb') as f:
            for data in downloadResponse.iter_content(chunk_size=8192):
                f.write(data)
        i+=1
        print(f"DOWNLOADED {filename} ({i}/{len(requestResults['availableDownloads'])})\n")
        



All downloads are available to download. Retrieving...

DOWNLOADING: https://landsatlook.usgs.gov/gen-bundle?landsat_product_id=LC08_L1TP_012054_20250401_20250411_02_T1&requestSignature=eyJkb3dubG9hZEFwcCI6Ik0yTSIsImNvbnRhY3RJZCI6MjY5MjQxNzgsImRvd25sb2FkSWQiOjc5MDcyMTExMywiZGF0ZUdlbmVyYXRlZCI6IjIwMjUtMDUtMDJUMDA6NTY6NTUtMDU6MDAiLCJpZCI6IkxDMDhfTDFUUF8wMTIwNTRfMjAyNTA0MDFfMjAyNTA0MTFfMDJfVDEiLCJzaWduYXR1cmUiOiIkNSQkWFdUWUZRbVZsY29LXC83MU9ycHdxSWZFdFNSSEhNV3A0bFlVSHFKVVN1ejkifQ==
DOWNLOADED LC08_L1TP_012054_20250401_20250411_02_T1.tar (1/14)

DOWNLOADING: https://landsatlook.usgs.gov/gen-browse?size=frb&type=refl&product_id=LC08_L1TP_012054_20250401_20250411_02_T1&requestSignature=eyJkb3dubG9hZEFwcCI6Ik0yTSIsImNvbnRhY3RJZCI6MjY5MjQxNzgsImRvd25sb2FkSWQiOjc5MDcyMTExNCwiZGF0ZUdlbmVyYXRlZCI6IjIwMjUtMDUtMDJUMDA6NTY6NTUtMDU6MDAiLCJpZCI6IkxDMDhfTDFUUF8wMTIwNTRfMjAyNTA0MDFfMjAyNTA0MTFfMDJfVDEiLCJzaWduYXR1cmUiOiIkNSQkWGEucDBiTVpjQmVWM3hreXRDelA3SjhjV2pub3dYYmtUZTFIT0pwY0xxRCJ9
DOWNLOADED LC08_L1

## **8. [*Logout*](https://m2m.cr.usgs.gov/api/docs/reference/#logout) of M2M Endpoint**<a id="logout-api"></a>

Logout so the API Key cannot be used anymore.

<div class="alert alert-block alert-warning">
    <h4> <b>NOTE:</b> The Machine-to-Machine API key expires after 2 hours of inactivity. </h4>
</div>  

In [38]:
endpoint = "logout"  
if sendRequest(serviceUrl + endpoint, None, apiKey) == None:        
    print("\n\nLogged Out\n")
else:
    print("\n\nLogout Failed\n")



Logged Out



## **9. List Downloads**

In [39]:
os.listdir(data_dir)

['LC09_L2SP_012054_20230319_20230321_02_T1.tar',
 'LC09_L2SP_012055_20230303_20230308_02_T1.tar',
 'LC09_L2SP_012055_20230319_20230321_02_T1.tar',
 'LC09_L2SP_013054_20230310_20230312_02_T1.tar']